# Imports

In [ ]:
try: 
    import casatools
except:
    !pip install casatools -i https://casa-pip.nrao.edu:443/repository/pypi-group/simple
            
try:
    import casatasks
except:
    !pip install casatasks -i https://casa-pip.nrao.edu:443/repository/pypi-group/simple

try:
    import casatestutils
except:
    !pip install casatestutils -i https://casa-pip.nrao.edu:443/repository/pypi-group/simple

In [ ]:
import os, sys
import unittest
import pytest
import shutil

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)

In [ ]:
user_data_path = os.path.expanduser("~/.casa/measures")
exec_path = sys.executable
IN_COLAB = 'google.colab' in sys.modules

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
print("In Google Colab: {}".format(IN_COLAB))

In [ ]:
 if not os.path.exists(os.path.expanduser("~/.casa/measures")):
    !{exec_path} -m casatools --update-user-data

# Checkout Version of runtest.py to test

In [ ]:
if IN_COLAB:
    %cd /content
    pass
else:
    workpath = cwd + '/work'
    if os.path.exists(workpath):
        shutil.rmtree(workpath)
    os.makedirs(workpath)
    %cd {workpath}
!pwd 

In [ ]:
!git clone https://open-bitbucket.nrao.edu/scm/casa/casa6.git
    
### Uncomment to Test Specific Version of Runtest From Branch
#%cd casa6
#!git checkout CAS-13789
#if IN_COLAB:
#    %cd /content
#else:
#    os.chdir("..")

## Sparse Checkout Data

In [ ]:
if IN_COLAB:
    !sudo apt-get install git-lfs

In [ ]:
if IN_COLAB:
    %cd /content/
    !rm -rf casatestdata/ 
else:
    datapath = "/home/casa/data/casatestdata/"

In [ ]:
if IN_COLAB:
    !git clone --no-checkout https://open-bitbucket.nrao.edu/scm/casa/casatestdata.git
    %cd /content/casatestdata
    !git config core.sparseCheckout true
    !git config filter.lfs.clean "git-lfs clean -- %f"
    !git config filter.lfs.smudge "git-lfs smudge -- %f"
    !git config filter.lfs.process "git-lfs filter-process" 
    !echo "unittest/*" >> .git/info/sparse-checkout 
    #!echo "image/*" >> .git/info/sparse-checkout 
    !echo "measurementset/alma/nep2-shrunk.ms/*" >> .git/info/sparse-checkout 
    !git checkout master

In [ ]:
!git config --list 

In [ ]:
!pwd
!ls -alh

In [ ]:
if IN_COLAB:
    !rm -rf /root/.casa/config.py
    !touch /root/.casa/config.py
    !echo "import time" >> /root/.casa/config.py
    !echo "import sys" >> /root/.casa/config.py
    !echo "datapath=[ '/content/casatestdata/']" >> /root/.casa/config.py
    !cat /root/.casa/config.py
else:
    config_path = os.path.expanduser("~/.casa/config.py")
    if os.path.exists(config_path):
        !cat {config_path}
    else:
        !touch {config_path}
        !echo "import time" >> {config_path}
        !echo "import sys" >> {config_path}
        string = "datapath=[ '{}']".format(datapath)
        !echo {string} >> {config_path}

# Tests

## Test Case 1

Test that runtest finds working path

In [ ]:
!pwd

In [ ]:
class TestClass(unittest.TestCase):
      
    @classmethod
    def setUpClass(cls):
        !touch test_foo.py

    @classmethod
    def tearDownClass(cls):
        !rm -rf nosedir
        !rm -rf test_foo.py

    def test_nosedir_created(self):
        if IN_COLAB:
            !{exec_path} /content/casa6/casatestutils/runtest.py test_foo.py
        else:
            !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py test_foo.py
        self.assertTrue(os.path.exists("nosedir/"))


## Test Case 2

Test that runtest creates test working directory 

In [ ]:
class TestClass2(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        !touch test_foo.py

    @classmethod
    def tearDownClass(cls):
        !rm -rf nosedir
        !rm -rf test_foo.py

    def test_nosedir_test_directory_created(self):
        if IN_COLAB:
            !{exec_path} /content/casa6/casatestutils/runtest.py test_foo.py
        else:
            !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py test_foo.py
        self.assertTrue(os.path.exists("nosedir/test_foo"))

## Test Case 3

Test that runtest copies local test to working directory

In [ ]:
class TestClass3(unittest.TestCase):
  
    @classmethod
    def setUpClass(cls):
        !touch test_foo.py

    @classmethod
    def tearDownClass(cls):
        !rm -rf nosedir
        !rm -rf test_foo.py

    def test_nosedir_test_copied(self):
        if IN_COLAB:
            !{exec_path} /content/casa6/casatestutils/runtest.py test_foo.py
        else:
            !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py test_foo.py
        self.assertTrue(os.path.exists("nosedir/test_foo/test_foo.py"))

## Test Case 4

Test git cloned test repositories


In [ ]:
class TestClass4(unittest.TestCase):

    @classmethod
    def tearDownClass(cls):
        !rm -rf nosedir
    
    def test_nosedir_test_copied(self):
        if IN_COLAB:
            !{exec_path} /content/casa6/casatestutils/runtest.py -x test_task_tclean
        else:
            !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py -x test_task_tclean
        self.assertTrue(os.path.exists("nosedir/casasources/almatasks"))
        self.assertTrue(os.path.exists("nosedir/casasources/casa6"))
        self.assertTrue(os.path.exists("nosedir/casasources/casampi"))
        self.assertTrue(os.path.exists("nosedir/casasources/casaplotms"))
        self.assertTrue(os.path.exists("nosedir/casasources/casaviewer"))

## Test Case 5a

Test that runtest runs a casatool test script from git

In [ ]:
if IN_COLAB:
    !{exec_path}  /content/casa6/casatestutils/runtest.py -x test_tool_componentlist
else:
    !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py -x test_tool_componentlist

## Test Case 5b - optional

Test that runtest runs a casatask test script from git

In [ ]:
### Run If Need Be
#if IN_COLAB:
#    %cd /content/casa6/casatestutils
#    !{exec_path} runtest.py test_task_plotweather
#    %cd /content/
#else:
#    %cd {cwd}/work/casa6/casatestutils
#    !{exec_path} runtest.py test_task_plotweather
#    %cd {cwd}

# Test Case 6 

run a single test case


In [ ]:
if IN_COLAB:
    !{exec_path} /content/casa6/casatestutils/runtest.py test_tool_componentlist[test_plp]
else:
    !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py test_tool_componentlist[test_plp]


# Test Case 7

run two test cases

In [ ]:
if IN_COLAB:
    %cd /content/casa6/casatestutils
    !{exec_path} runtest.py test_tool_componentlist[test_plp,test_summarize,test_getfluxerror]
    %cd /content
else:
    %cd {cwd}/work/casa6/casatestutils/
    !{exec_path} runtest.py test_tool_componentlist[test_plp,test_summarize,test_getfluxerror]
    %cd {cwd}

# Test Case 8

run test using --ignore-list

In [ ]:
class TestClass8(unittest.TestCase):
    
    @classmethod
    def tearDownClass(cls):
        !rm -rf nosedir

    def test_ignore_list(self):
        if IN_COLAB:
            !{exec_path} /content/casa6/casatestutils/runtest.py --ignore_list=test_task_tclean test_tool_componentlist,test_task_tclean
        else:
            !{exec_path} {cwd}/work/casa6/casatestutils/runtest.py --ignore_list=test_task_tclean test_tool_componentlist,test_task_tclean
        self.assertTrue(os.path.exists("nosedir/test_tool_componentlist"))
        self.assertFalse(os.path.exists("nosedir/test_task_tclean"))

# Test Case 9
Run two test scripts selecting one test case from each one of them

In [ ]:
if IN_COLAB:
    %cd /content/casa6/casatestutils
    !{exec_path} runtest.py test_tool_componentlist[test_plp],test_tool_image[test_general]
    %cd /content
else:
    %cd {cwd}/work/casa6/casatestutils/
    !{exec_path} runtest.py test_tool_componentlist[test_plp],test_tool_image[test_general]
    %cd {cwd}

# Test 10
Run two test script, one with two test cases, the other in full

In [ ]:
if IN_COLAB:
    %cd /content/casa6/casatestutils
    !{exec_path} runtest.py test_tool_image[test_general,test_stretch],test_tool_componentlist
    %cd /content
else:
    %cd {cwd}/work/casa6/casatestutils/
    !{exec_path} runtest.py test_tool_image[test_general,test_stretch],test_tool_componentlist
    %cd {cwd}

# Test 11

In [ ]:
if IN_COLAB:
    %cd /content/casa6/casatestutils
    !{exec_path} runtest.py test_tool_componentlist,test_tool_image[test_general,test_stretch]
    %cd /content
else:
    %cd {cwd}/work/casa6/casatestutils/
    !{exec_path} runtest.py test_tool_componentlist,test_tool_image[test_general,test_stretch]
    %cd {cwd}

# Test 12

In [ ]:
if IN_COLAB:
    %cd /content/casa6/casatestutils
    !{exec_path} runtest.py -x -j fixplanets
    %cd /content
else:
    %cd {cwd}/work/casa6/casatestutils/
    !{exec_path} runtest.py -x -j fixplanets
    %cd {cwd}

# Run Tests

In [ ]:
res = unittest.main(argv=[''], verbosity=3, exit=False)

# Clean Up

In [ ]:
if not IN_COLAB:
    !rm -rf {cwd}/work